# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install "openai<1.0.0"

  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
Using cached openai-0.28.1-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0


In [2]:
import os
import openai
import re
openai.api_type = "azure"
openai.api_base = "https://pnlp-janseen.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = OPENAI_API_KEY
import pandas as pd
import mlflow
from mlflow.metrics.metric_definitions import _token_count_eval_fn

In [3]:
experiment_name = "MAS"
# mlflow.create_experiment(experiment_name)#, artifact_location="s3://my-byte/Joel/ByteMLFlow")
mlflow.set_experiment(experiment_name)

2024/03/26 10:22:21 INFO mlflow.tracking.fluent: Experiment with name 'MAS' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///newvolume/mybyte/Joel/Byte/mlruns/644831852993414526', creation_time=1711448541124, experiment_id='644831852993414526', last_update_time=1711448541124, lifecycle_stage='active', name='MAS', tags={}>

In [4]:
Data = pd.read_excel("content/Childern_HA551.xlsx")
# Data = pd.read_excel("/content/Other334 (4).xlsx")
#Data = pd.read_csv("/content/338_dec11.csv", encoding='cp1252')
Data = Data.reset_index(drop=True)
Data = Data.fillna("No-Data")
dict_list = Data.to_dict(orient='records')
dict_list[2].keys()

dict_keys(['Author(Publication_Year)', 'Full_text', 'study_design_sent', 'study_design', 'control_sent', 'control', 'blinding_sent', 'blinding', 'randomization_method_sent', 'randomization_method', 'aim', 'population_sent', 'population', 'diagnosis_of_interest_sent', 'diagnosis_of_interest', 'diagnostic_criteria_sent', 'diagnostic_criteria', 'age_sent', 'age', 'sex_sent', 'sex', 'treatment_sent', 'treatment', 'disease_condition_sent', 'disease_condition', 'treatment_duration_sent', 'treatment_duration', 'enrolled_sent', 'enrolled', 'analyzed_total_sent', 'analyzed_total', 'analyzed_efficacy_sent', 'analyzed_efficacy', 'country_sent', 'country', 'efficacy_results', 'safety_results', 'efficacy_endpoints', 'safety_endpoints', 'author_conclusion', 'by_treatment_sent', 'by_treatment', 'dose_sent', 'dose', 'dosage_form_sent', 'dosage_form', 'dose_frequency_sent', 'dose_frequency', 'mode_of_administration_sent', 'mode_of_administration', 'analyzed_safety_sent', 'analyzed_safety'])

In [5]:
whole_data_entity1= "\n\n".join(["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)', 'study_design_sent', 'study_design', 'control_sent', 'control', 'blinding_sent', 'blinding', 'randomization_method_sent', 'randomization_method', 'aim', 'population_sent', 'population', 'diagnosis_of_interest_sent', 'diagnosis_of_interest', 'diagnostic_criteria_sent', 'diagnostic_criteria', 'age_sent', 'age', 'sex_sent', 'sex', 'treatment_sent', 'treatment', 'disease_condition_sent', 'disease_condition', 'treatment_duration_sent', 'treatment_duration', 'enrolled_sent', 'enrolled', 'analyzed_total_sent', 'analyzed_total', 'analyzed_efficacy_sent', 'analyzed_efficacy', 'country_sent', 'country', 'efficacy_results', 'safety_results', 'efficacy_endpoints', 'safety_endpoints', 'author_conclusion', 'by_treatment_sent', 'by_treatment', 'dose_sent', 'dose', 'dosage_form_sent', 'dosage_form', 'dose_frequency_sent', 'dose_frequency', 'mode_of_administration_sent', 'mode_of_administration', 'analyzed_safety_sent', 'analyzed_safety']}.items() if value != 'No-Data']) for dictionary in dict_list])

In [7]:
objective_data = "\n\n".join(["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)', 'aim', 'diagnosis_of_interest', 'diagnosis_of_interest_sent', 'diagnostic_criteria', 'diagnostic_criteria_sent', 'disease_condition', 'disease_condition_sent', 'treatment', 'treatment_sent', 'efficacy_results', 'safety_results','author_conclusion']}.items() if value != 'No-Data']) for dictionary in dict_list])
Conclusion_data = "\n\n".join(['\n'.join([f'{key}:{value}' for key, value in {key: dictionary[key] for key in ['Author(Publication_Year)','population','population_sent','enrolled','enrolled_sent','analyzed_total','analyzed_total_sent','analyzed_efficacy','analyzed_efficacy_sent','efficacy_endpoints','efficacy_results','analyzed_safety','analyzed_safety_sent','safety_endpoints','safety_results','by_treatment','by_treatment_sent','author_conclusion']}.items() if value != 'No-Data']) for dictionary in dict_list])
Study_design_Dosage_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','study_design', 'study_design_sent', 'randomization_method', 'randomization_method_sent', 'control', 'control_sent', 'blinding', 'blinding_sent','age', 'age_sent', 'sex', 'sex_sent','population', 'population_sent','enrolled', 'enrolled_sent','dose', 'dosage_form', 'dosage_form_sent', 'dose_frequency', 'dose_frequency_sent', 'mode_of_administration', 'mode_of_administration_sent', 'treatment', 'treatment_sent','treatment_duration', 'treatment_duration_sent']] for dictionary in dict_list]]]
Efficacy_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)', 'analyzed_efficacy', 'analyzed_efficacy_sent','efficacy_endpoints','efficacy_results','by_treatment','by_treatment_sent']] for dictionary in dict_list]]]
Safety_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','analyzed_safety','analyzed_safety_sent','safety_endpoints','safety_results']] for dictionary in dict_list]]]
Ojective_split_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)', 'aim', 'diagnosis_of_interest', 'diagnosis_of_interest_sent']] for dictionary in dict_list]]]
Conclusion_split_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','population','population_sent','enrolled','enrolled_sent','analyzed_total','analyzed_total_sent','analyzed_efficacy','analyzed_efficacy_sent','efficacy_endpoints','efficacy_results','analyzed_safety','analyzed_safety_sent','safety_endpoints','safety_results','by_treatment','by_treatment_sent','author_conclusion']] for dictionary in dict_list]]]
Population_data = "\n\n".join(['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','age', 'age_sent', 'sex', 'sex_sent','population', 'population_sent','analyzed_efficacy', 'analyzed_efficacy_sent','efficacy_endpoints','efficacy_results','by_treatment','by_treatment_sent','analyzed_safety_sent','safety_endpoints','safety_results']] for dictionary in dict_list]]])
Dosage_data =  "\n\n".join(['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','dose', 'dosage_form', 'dosage_form_sent', 'dose_frequency', 'dose_frequency_sent', 'mode_of_administration', 'mode_of_administration_sent', 'treatment_duration', 'treatment_duration_sent']] for dictionary in dict_list]]])
Result_data = ['\n'.join(inner_list) for inner_list in [[item for item in sublist if ":No-Data" not in item] for sublist in [[f'{key}:{dictionary[key]}' for key in ['Author(Publication_Year)','population', 'population_sent','enrolled', 'enrolled_sent', 'analyzed_efficacy', 'analyzed_efficacy_sent','efficacy_endpoints','efficacy_results','by_treatment','by_treatment_sent','safety_results']] for dictionary in dict_list]]]


In [8]:
objective_data_list = ["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)', 'aim', 'diagnosis_of_interest', 'diagnosis_of_interest_sent', 'diagnostic_criteria', 'diagnostic_criteria_sent', 'disease_condition', 'disease_condition_sent', 'treatment', 'treatment_sent', 'efficacy_results', 'safety_results','author_conclusion']}.items() if value != 'No-Data']) for dictionary in dict_list]
full_text = ["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)','Full_text']}.items() if value != 'No-Data']) for dictionary in dict_list]
whole_data = "\n\n".join(["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)','study_design_sent', 'study_design', 'control_sent', 'control', 'blinding_sent', 'blinding', 'randomization_method_sent', 'randomization_method', 'aim', 'population_sent', 'population', 'diagnosis_of_interest_sent', 'diagnosis_of_interest', 'diagnostic_criteria_sent', 'diagnostic_criteria', 'age_sent', 'age', 'sex_sent', 'sex', 'treatment_sent', 'treatment', 'disease_condition_sent', 'disease_condition', 'treatment_duration_sent', 'treatment_duration', 'enrolled_sent', 'enrolled', 'analyzed_total_sent', 'analyzed_total', 'analyzed_efficacy_sent', 'analyzed_efficacy', 'country_sent', 'country', 'efficacy_results', 'safety_results', 'efficacy_endpoints', 'safety_endpoints', 'author_conclusion', 'by_treatment_sent', 'by_treatment', 'dose_sent', 'dose', 'dosage_form_sent', 'dosage_form', 'dose_frequency_sent', 'dose_frequency', 'mode_of_administration_sent', 'mode_of_administration', 'analyzed_safety_sent', 'analyzed_safety']}.items() if value != 'No-Data']) for dictionary in dict_list])
whole_data_entity = "\n\n".join(["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)','study_design_sent', 'study_design', 'control_sent', 'control', 'blinding_sent', 'blinding', 'randomization_method_sent', 'randomization_method', 'aim', 'population_sent', 'population', 'diagnosis_of_interest_sent', 'diagnosis_of_interest', 'diagnostic_criteria_sent', 'diagnostic_criteria', 'age_sent', 'age', 'sex_sent', 'sex', 'treatment_sent', 'treatment', 'disease_condition_sent', 'disease_condition', 'treatment_duration_sent', 'treatment_duration','efficacy_results', 'safety_results','author_conclusion','dose', 'dosage_form', 'dose_frequency','mode_of_administration']}.items() if value != 'No-Data']) for dictionary in dict_list])
whole_data_entity_list = ["\n".join([f"{key}:{value}" for key, value in {key: dictionary[key] for key in dictionary if key in ['Author(Publication_Year)','study_design_sent', 'study_design', 'control_sent', 'control', 'blinding_sent', 'blinding', 'randomization_method_sent', 'randomization_method', 'aim', 'population_sent', 'population', 'diagnosis_of_interest_sent', 'diagnosis_of_interest', 'diagnostic_criteria_sent', 'diagnostic_criteria', 'age_sent', 'age', 'sex_sent', 'sex', 'treatment_sent', 'treatment', 'disease_condition_sent', 'disease_condition', 'treatment_duration_sent', 'treatment_duration','efficacy_results', 'safety_results','author_conclusion','dose', 'dosage_form', 'dose_frequency','mode_of_administration']}.items() if value != 'No-Data']) for dictionary in dict_list]

In [9]:
def extract_MAS_single(prompt):

    response = openai.ChatCompletion.create(
        messages = [
            {"role": "user", "content": prompt}
            ],
            temperature = 0,
            max_tokens = 4000,
            engine = "GPT4Turbo1106Preview",
            # model = "gpt-3.5-turbo-16k",
            api_key = OPENAI_API_KEY,
        )
    return response["choices"][0]["message"]['content']

In [10]:
import time
prompt_1 =f""""Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary with in-text citations and the summary should be minimum 1000 words, following the structure provided below:

    Summary Structure:
    The first paragraph should briefly outline the common objective, population range across the studies, study design across all the studies, common intervention with dose and dosage, endpoints measured across all the studies, and studies that do not align with the common intervention of all referenced articles. (Very Important: Objective Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)
    The second paragraph should focus on comparing and contrasting the efficacy results of all primary and secondary endpoints, mentioning the treatment number of subjects, treatment duration, and dosage with citations. If the end results of articles are the same, provide a end result and cite all the articles that fall under that must include efficacy of the medical articles under scrutiny and their percentages.(Very Important: Efficacy Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)
    The third paragraph should concentrate on comparing safety and adverse events across all studies with citation. It's crucial to include patient counts and cases across the studies. (Very Important: safety Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token)

    The final paragraph should present the overall conclusion of the findings, along with a risk-benefit analysis. (Very Important: Conclusion Section should be elaborative and quantitative with the length of paragraph to be minimum 1000 token) 

    Rule to follow: Ensure that facts are cited by Author (Publication_Year) of the corresponding studies.

    Data of multiple clinical research articles is delimited by triple backticks \n```{whole_data_entity}```\n"""
# start = time.time()
# response_1 = extract_MAS_single(prompt_1)
# duration = time.time() - start
# print(response_1)

In [11]:
with mlflow.start_run(run_name='check_1',
                      tags={"mlflow.source.git.commit": 1.0},
                      log_system_metrics=True):
    response_1 = extract_MAS_single(prompt_1)

    mlflow.log_text(prompt_1, "prompt.txt")
    mlflow.log_text(response_1, "summary.txt")
    mlflow.log_metric("prompt_token", _token_count_eval_fn([whole_data_entity]).scores[0])
    mlflow.log_metric("summary_token", _token_count_eval_fn([response_1]).scores[0])

2024/03/26 10:23:41 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


2024/03/26 10:24:42 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/03/26 10:24:42 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [17]:
_token_count_eval_fn([response_1]).scores[0]

853

In [ ]:
# prompt_1 =f"""Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary with in-text citations and following the structure provided below:

# Summary Structure:
# The first paragraph should briefly outline the common objective, population range across the studies, study design across all the studies, common intervention with dose and dosage, endpoints measured across all the studies, and studies that do not align with the common intervention of all referenced articles.
# The second paragraph should focus on comparing and contrasting the efficacy results of all primary and secondary endpoints, mentioning the treatment number of subjects, treatment duration, and dosage with citations. If the end results of articles are the same, provide a end result and cite all the articles that fall under that must include efficacy of the medical articles under scrutiny and their percentages.(Very Important: Efficacy Section should be elaborative and quantitative with more than 200 tokens)
# The third paragraph should concentrate on comparing safety and adverse events across all studies with citation. It's crucial to include patient counts and cases across the studies.

# The final paragraph should present the overall conclusion of the findings, along with a risk-benefit analysis. 

# Rule to follow: Ensure that facts are cited by Author (Publication_Year) of the corresponding studies.

# Data of multiple clinical research articles is delimited by triple backticks \n```{whole_data_entity1}```\n"""

In [ ]:
# prompt_1 =f"""
# Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary following the structure provided below:

# Summary Structure:

# The first paragraph should briefly outline the common objective, population range across the studies, study design across all the studies, common intervention, and studies that do not align with the common intervention of all referenced articles.

# The second paragraph should focus on Comparing the efficacy results of primary endpoints across all medical articles, focusing on treatment number of subjects, treatment duration, and dosage with proper citations. If multiple articles report identical end results, provide an efficacy result and cite all articles sharing that outcome, including their respective percentages. Ensure all data is available for comparative analysis.

# The third paragraph should concentrate on comparing safety and adverse events across all studies with citations. It's crucial to include patient counts and cases across the studies along with the percentages. If the end results of articles are the same, provide a Safety result and cite all the articles that fall under that, including the safety of the medical articles under scrutiny and their percentages.

# The final paragraph should present the overall conclusion of the findings, along with a risk-benefit analysis. 

# Rule to follow: Ensure that facts are cited by Author (Publication_Year) of the corresponding studies.

# The medical article is delimited by triple backticks ```{whole_data_entity1}```.
# """
# response_1 = extract_MAS_single(prompt_1)
# print(response_1)

In [ ]:
prompt_1 = f"""
Act as a medical review writer and review the given data from multiple clinical research articles. Write a factual summary following the structure provided below:

Summary Structure:

The first paragraph should briefly outline the common objective, population range across the studies, study design, common intervention, and studies that do not align with the common intervention of all referenced articles.

The second paragraph should focus on the efficacy results of all primary endpoints, mentioning the treatment number of subjects, treatment duration, and dosage.

The third paragraph should concentrate on comparing safety and adverse events across all studies. It's crucial to include patient counts and cases across the studies.

The final paragraph should present the overall conclusion of the findings, along with a risk-benefit analysis.

Rule to follow: Ensure that facts are cited by Author (Publication_Year) of the corresponding studies.

The medical article is delimited by triple backticks ```{whole_data_entity1}```.
"""


In [ ]:
"""Act as a medical review writer, review the given data of multiple clinical research articles and write a factual summary as best as possible in the following structure.

Summary Structure:

The first paragraph should brief the common objective, population range across the studies, study design, common intervention and studies which are not aligning to common intervention of all articles referenced.

Second paragraph should focus on the efficacy results of all the primary endpoints, mentioning the, treatment number of subjects, treatment duration and dosage

Third paragraph should focus on a comparison of safety and adverse events across all studies (Very Important: Make sure to include patient's counts and cases across the studies)

The final paragraph with the overall conclusion of the findings with risk benefit analysis.

Rule to follow: Make sure to cite the facts by Author(Pubication_Year) of corresponding studies."""